In [11]:
import os
from datetime import datetime, timedelta
import pandas as pd
import gspread
import warnings
warnings.filterwarnings('ignore')

def datetime_to_string(date):
    """날짜타입에서 문자열로 변경"""
    return date.strftime('%Y-%m-%d')

def weekday_to_string(weekday_number):
    """요일 숫자를 문자로 변경하는 함수"""
    if weekday_number == 0:
        return "월요일"
    elif weekday_number == 1:
        return "화요일"
    elif weekday_number == 2:
        return "수요일"
    elif weekday_number == 3:
        return "목요일"
    elif weekday_number == 4:
        return "금요일"
    elif weekday_number == 5:
        return "토요일"
    elif weekday_number == 6:
        return "일요일"
    else:
        return ""
    
def date_diff_days(start_date, end_date):
    """두 날짜의 차이를 구하는 함수"""
    start_datetime = datetime.strptime(start_date, '%Y-%m-%d')
    end_datetime = datetime.strptime(end_date, '%Y-%m-%d')
    date_diff = end_datetime - start_datetime
    date_diff = date_diff + timedelta(days=1)
    return date_diff.days

file_path = os.path.join(os.environ["HOMEPATH"], "Downloads/")

# 기준날짜와 비교날짜 설정
today = datetime.now()
#today = today + timedelta(days=-1)
today = today + timedelta(days=-2)
yesterday = today + timedelta(days=-1)
last_yesterday = today + timedelta(days=-8)
last_week = yesterday + timedelta(days=-6)
last_month = yesterday + timedelta(days=-29)

yesterday = datetime_to_string(yesterday)
last_yesterday = datetime_to_string(last_yesterday)
last_week = datetime_to_string(last_week)
last_month = datetime_to_string(last_month)

print("기준날짜는", yesterday, "입니다.")
print("최근 일주일 날짜는", last_week, "입니다.")
print("최근 한달 날짜는", last_month, "입니다.")


# 원본 테이블 불러오기
sa = gspread.service_account(f"{file_path}snappy-cosine-411501-fbfbf5c109c9.json")
sh = sa.open("레드테이블x아이오로라")
wks = sh.worksheet("외부인원_NEW_결제내역")
values = wks.get_all_values()
header, rows = values[0], values[1:]
df_all_card = pd.DataFrame(rows, columns=header)

df_all_card["결제날짜"] = df_all_card["결제날짜"].apply(lambda x:x.replace("-",""))
df_all_card["결제날짜"] = df_all_card["결제날짜"].apply(lambda x:x[:4] + "-" + x[4:6] + "-" + x[6:8])

# 결제금액 컬럼의 데이터 타입 변환
df_all_card["결제금액"] = df_all_card["결제금액"].apply(lambda x:int(x.replace(",", "").replace("₩", "")))
df_all_card["요일"] = df_all_card["결제날짜"].apply(lambda x:datetime.strptime(x, "%Y-%m-%d").weekday())

기준날짜는 2024-04-14 입니다.
최근 일주일 날짜는 2024-04-08 입니다.
최근 한달 날짜는 2024-03-16 입니다.


In [17]:
start_date = last_month
end_date = yesterday

def week_people_date(end_date):
    """요일 단위로 사용자들을 구하는 함수"""
    day_week = df_all_card[df_all_card["결제날짜"] == end_date]["요일"].unique()[0]
    
    df_date = df_all_card[df_all_card["요일"] == day_week]
    unique_remarks = df_date["채널 대분류"].unique()
    df_summary = pd.DataFrame()
    
    for people in unique_remarks:
        # 현재 비고 값에 대한 데이터만 필터링
        filtered_df = df_date[df_date["채널 대분류"] == people]

        # 결제날짜별로 고유 사용자 수 집계
        grouped = filtered_df.groupby("결제날짜").agg(
            **{people: pd.NamedAgg(column="iaurora_id", aggfunc="nunique")}
        ).reset_index()

        # 집계된 데이터를 df_summary에 병합
        if df_summary.empty:
            df_summary = grouped
        else:
            df_summary = pd.merge(df_summary, grouped, on="결제날짜", how="outer")

    df_summary = df_summary.fillna(0)
    df_summary.sort_values(by=["결제날짜"], ascending=[True], inplace=True)
    df_summary.reset_index(drop=True, inplace=True)
    df_summary.loc[:, "전체"] = df_summary.loc[0:len(df_summary)-1, :].sum(axis=1)
    df_summary.loc["평균", :len(df_summary.columns)] = df_summary.loc[0:len(df_summary)-1, :].mean(axis=0).round(1)
    df_summary.loc["평균", "결제날짜"] = f"{weekday_to_string(day_week)}"
    return df_summary[-5:]


def week_pay_date(end_date):
    """요일 단위로 사용금액을 구하는 함수"""
    day_week = df_all_card[df_all_card["결제날짜"] == end_date]["요일"].unique()[0]

    df_date = df_all_card[df_all_card["요일"] == day_week]
    unique_remarks = df_date["채널 대분류"].unique()
    df_summary = pd.DataFrame()

    for people in unique_remarks:
        # 현재 비고 값에 대한 데이터만 필터링
        filtered_df = df_date[df_date["채널 대분류"] == people]

        # 결제날짜별로 고유 사용자 수 집계
        grouped = filtered_df.groupby("결제날짜").agg(
            **{people: pd.NamedAgg(column="결제금액", aggfunc="sum")}
        ).reset_index()

        # 집계된 데이터를 df_summary에 병합
        if df_summary.empty:
            df_summary = grouped
        else:
            df_summary = pd.merge(df_summary, grouped, on="결제날짜", how="outer")

    df_summary = df_summary.fillna(0)
    df_summary.sort_values(by=["결제날짜"], ascending=[True], inplace=True)
    df_summary.reset_index(drop=True, inplace=True)
    df_summary.loc[:, "전체"] = df_summary.loc[0:len(df_summary)-1, :].sum(axis=1)
    df_summary.loc["평균", :len(df_summary.columns)] = df_summary.loc[0:len(df_summary)-1, :].mean(axis=0).round(0)
    df_summary.loc["평균", "결제날짜"] = f"{weekday_to_string(day_week)}"
    return df_summary[-5:]


def avg_people_date(start_date, end_date):
    """전체일자로 사용자들을 구하는 함수"""
    df_date = df_all_card[df_all_card["결제날짜"] >= start_date][df_all_card["결제날짜"] <= end_date]
    unique_remarks = df_date["채널 대분류"].unique()
    df_summary = pd.DataFrame()
    
    for people in unique_remarks:
        # 현재 비고 값에 대한 데이터만 필터링
        filtered_df = df_date[df_date["채널 대분류"] == people]

        # 결제날짜별로 고유 사용자 수 집계
        grouped = filtered_df.groupby("결제날짜").agg(
            **{people: pd.NamedAgg(column="iaurora_id", aggfunc="nunique")}
        ).reset_index()

        # 집계된 데이터를 df_summary에 병합
        if df_summary.empty:
            df_summary = grouped
        else:
            df_summary = pd.merge(df_summary, grouped, on="결제날짜", how="outer")
                                  
    df_summary = df_summary.fillna(0)
    df_summary.loc[:, "전체"] = df_summary.loc[0:len(df_summary)-1, :].sum(axis=1)
    df_summary.loc["평균", :len(df_summary.columns)] = df_summary.loc[0:len(df_summary)-1, :].mean(axis=0).round(1)
    df_summary.loc["평균", "결제날짜"] = f"최근 {date_diff_days(start_date, end_date)}일"
    return df_summary.loc[["평균"]]


def avg_pay_date(start_date, end_date):
    """전체일자로 결제금액들을 구하는 함수"""
    df_date = df_all_card[df_all_card["결제날짜"] >= start_date][df_all_card["결제날짜"] <= end_date]
    unique_remarks = df_date["채널 대분류"].unique()
    df_summary = pd.DataFrame()

    for people in unique_remarks:
        # 현재 비고 값에 대한 데이터만 필터링
        filtered_df = df_date[df_date["채널 대분류"] == people]

        # 결제날짜별로 고유 사용자 수 집계
        grouped = filtered_df.groupby("결제날짜").agg(
            **{people: pd.NamedAgg(column="결제금액", aggfunc="sum")}
        ).reset_index()

        # 집계된 데이터를 df_summary에 병합
        if df_summary.empty:
            df_summary = grouped
        else:
            df_summary = pd.merge(df_summary, grouped, on="결제날짜", how="outer")

    df_summary = df_summary.fillna(0)
    df_summary.loc[:, "전체"] = df_summary.loc[0:len(df_summary)-1, :].sum(axis=1)
    df_summary.loc["평균", :len(df_summary.columns)] = df_summary.loc[0:len(df_summary)-1, :].mean(axis=0).round(0)
    df_summary.loc["평균", "결제날짜"] = f"최근 {date_diff_days(start_date, end_date)}일"
    return df_summary.loc[["평균"]]


def category_date(start_date, end_date):
    """기준일자별로 카테고리를 구하는 함수"""
    df_date = df_all_card[df_all_card["결제날짜"] >= start_date][df_all_card["결제날짜"] <= end_date]
    grouped = df_date.groupby(["시/도", "시/군/구", "네이버_대분류"])
    df_summary = grouped.agg(
                총_결제금액=pd.NamedAgg(column="결제금액", aggfunc="sum"),
                결제수=pd.NamedAgg(column="iaurora_id", aggfunc="count"),
                결제인원=pd.NamedAgg(column="iaurora_id", aggfunc="nunique"),
                결제최대금액=pd.NamedAgg(column="결제금액", aggfunc="max")
            ).reset_index()
    df_summary.sort_values("결제수", ascending=False, inplace=True)
    df_summary["결제객단가"] = df_summary.apply(lambda row:(round(row["총_결제금액"]/row["결제인원"], 0)), axis=1).tolist()
    df_summary.reset_index(drop=True, inplace=True)
    return df_summary


def detail_category(df_all_card, yesterday, cate, area1, area2, mosu):
    """카테고리별 가장 많은 항목들 추출하는 함수"""
    df_date = df_all_card[df_all_card["결제날짜"] >= yesterday][df_all_card["결제날짜"] <= yesterday]
    grouped = df_date[(df_date["네이버_대분류"] == cate) & (df_date["시/도"] == area1) & (df_date["시/군/구"] == area2)].groupby(["시/도", "시/군/구", "상점명"])

    df_summary = grouped.agg(
                    총_결제금액=pd.NamedAgg(column="결제금액", aggfunc="sum"),
                    결제수=pd.NamedAgg(column="iaurora_id", aggfunc="count"),
                    결제인원=pd.NamedAgg(column="iaurora_id", aggfunc="nunique")
                ).reset_index()

    df_summary.sort_values("결제수", ascending=False, inplace=True)
    df_summary.reset_index(drop=True, inplace=True)
    return f"""{df_summary["결제수"][0]}건, {round((df_summary["결제수"][0]/mosu)*100, 2)}%, {df_summary["상점명"][0]}"""


def store_contract(end_date):
    """계약매장 파악하여 추출하는 함수"""
    store_contract = df_all_card[(df_all_card["결제날짜"] == yesterday) & (df_all_card["ktc_가맹여부"] == "1")]
    grouped = store_contract.groupby(["store_id", "상점명"])
    df_summary = grouped.agg(
                    총_결제금액=pd.NamedAgg(column="결제금액", aggfunc="sum"),
                    결제수=pd.NamedAgg(column="iaurora_id", aggfunc="count"),
                    결제인원=pd.NamedAgg(column="iaurora_id", aggfunc="nunique")
                ).reset_index()
    df_summary.sort_values("결제수", ascending=False, inplace=True)
    df_summary["결제객단가"] = df_summary.apply(lambda row:(round(row["총_결제금액"]/row["결제인원"], 0)), axis=1).tolist()
    df_summary.reset_index(drop=True, inplace=True)

    unique_remarks = store_contract["채널 대분류"].unique()
    df_summary2 = pd.DataFrame()

    for people in unique_remarks:
        # 현재 비고 값에 대한 데이터만 필터링
        filtered_df = store_contract[store_contract["채널 대분류"] == people]

        # 결제날짜별로 고유 사용자 수 집계
    #     grouped = filtered_df.groupby("store_id").agg(
    #         **{f"{people}_users": pd.NamedAgg(column="iorora_user_id", aggfunc="nunique"),
    #            f"{people}_total_amount": pd.NamedAgg(column="결제금액", aggfunc="sum")}
    #     ).reset_index()

        grouped = filtered_df.groupby("store_id").agg(
            **{f"{people}": pd.NamedAgg(column="iaurora_id", aggfunc="nunique")}
        ).reset_index()

        # 집계된 데이터를 df_summary에 병합
        if df_summary2.empty:
            df_summary2 = grouped
        else:
            df_summary2 = pd.merge(df_summary2, grouped, on="store_id", how="outer")

    return pd.merge(df_summary, df_summary2, on="store_id")


def int_to_comma(number):
    """숫자 3자리씩 콤마를 넣는 함수"""
    return format(int(number), ',')


def comma_to_int(comma_number):
    """숫자 안에 있는 콤마를 제외해서 숫자형태로 변환하는 함수"""
    return int(comma_number.replace(",",""))


def variance(first,second):
    """비교하여 증감을 표기하는 함수"""
    if isinstance(first, str):
        first = comma_to_int(first)
    if isinstance(second, str):
        second = comma_to_int(second)
        
    if first - second > 0:
        return "+ " + str("{:,}".format(round(first-second, 1))) + " 증가"
    elif first - second < 0:
        return str("{:,}".format(round(first-second, 1))) + " 감소"
    else:
        return "+" + str(round(first-second, 1)) + " "

In [18]:
# 날짜 파싱 한 번만 수행
parsed_date = datetime.strptime(yesterday, "%Y-%m-%d")
year, month, day = parsed_date.year, parsed_date.month, parsed_date.day

day_week = df_all_card[df_all_card["결제날짜"] == end_date]["요일"].unique()[0]

########### 1) 사용인원 #############
week_people_data = week_people_date(yesterday)
standard_people_data = week_people_data[week_people_data["결제날짜"]==yesterday].reset_index(drop=True)
last_yesterday_people_data = week_people_data[week_people_data["결제날짜"]==last_yesterday].reset_index(drop=True)
last_week_people_data = avg_people_date(last_week, yesterday)
last_month_people_data = avg_people_date(last_month, yesterday)

standard_people_total = standard_people_data["전체"][0]
last_yesterday_people_total = last_yesterday_people_data["전체"][0]
last_week_people_total = last_week_people_data["전체"][0]
last_month_people_total = last_month_people_data["전체"][0]

# 카테고리별 차이 계산
diff_people_ctrip = standard_people_data["씨트립"][0] - last_yesterday_people_data["씨트립"][0]
diff_people_hanyumang = standard_people_data["한유망"][0] - last_yesterday_people_data["한유망"][0]
diff_people_student = standard_people_data["유학생"][0] - last_yesterday_people_data["유학생"][0]
diff_people_internal = standard_people_data["내부인원"][0] - last_yesterday_people_data["내부인원"][0]

diff_people_ctrip2 = round(standard_people_data["씨트립"][0] - last_week_people_data["씨트립"][0], 1)
diff_people_hanyumang2 = round(standard_people_data["한유망"][0] - last_week_people_data["한유망"][0], 1)
diff_people_student2 = round(standard_people_data["유학생"][0] - last_week_people_data["유학생"][0], 1)
diff_people_internal2 = round(standard_people_data["내부인원"][0] - last_week_people_data["내부인원"][0], 1)

diff_people_ctrip3 = round(standard_people_data["씨트립"][0] - last_month_people_data["씨트립"][0], 1)
diff_people_hanyumang3 = round(standard_people_data["한유망"][0] - last_month_people_data["한유망"][0], 1)
diff_people_student3 = round(standard_people_data["유학생"][0] - last_month_people_data["유학생"][0], 1)
diff_people_internal3 = round(standard_people_data["내부인원"][0] - last_month_people_data["내부인원"][0], 1)

########### 2) 사용금액 #############
week_pay_data = week_pay_date(yesterday)
standard_pay_data = week_pay_data[week_pay_data["결제날짜"]==yesterday].reset_index(drop=True)
last_yesterday_pay_data = week_pay_data[week_pay_data["결제날짜"]==last_yesterday].reset_index(drop=True)
last_week_pay_data = avg_pay_date(last_week, yesterday)
last_month_pay_data = avg_pay_date(last_month, yesterday)

standard_pay_total = int_to_comma(standard_pay_data["전체"][0])
last_yesterday_pay_total = int_to_comma(last_yesterday_pay_data["전체"][0])
last_week_pay_total = int_to_comma(last_week_pay_data["전체"][0])
last_month_pay_total = int_to_comma(last_month_pay_data["전체"][0])

########### 3) 분류별 #############
category_all_data = category_date(yesterday, yesterday)
category_all_data["지역"] = category_all_data.apply(lambda row:(row["시/도"]+ " "+ row["시/군/구"]), axis=1).tolist()

category_list = ["쇼핑비", "음식점", "문화/오락비", "의료/뷰티비", "기타"]

KeyError: '시/도'

In [4]:
print(f"""
{year}년 {month}월 {day}일 코리아트래블 카드현황

1) 사용인원

- 기준일평균 : {standard_people_total}명 (씨트립 {standard_people_data["씨트립"][0]} + 한유망 {standard_people_data["한유망"][0]} + 유학생 {standard_people_data["유학생"][0]} + 내부인원 {standard_people_data["내부인원"][0]})
- 전체{weekday_to_string(day_week)}평균 : {week_people_data.loc["평균":]["전체"][0]}명 (씨트립 {week_people_data.loc["평균":]["씨트립"][0]} + 한유망 {week_people_data.loc["평균":]["한유망"][0]} + 유학생 {week_people_data.loc["평균":]["유학생"][0]} + 내부인원 {week_people_data.loc["평균":]["내부인원"][0]})
- 지난{weekday_to_string(day_week)}평균 : {last_yesterday_people_total}명 (씨트립 {last_yesterday_people_data["씨트립"][0]} + 한유망 {last_yesterday_people_data["한유망"][0]} + 유학생 {last_yesterday_people_data["유학생"][0]} + 내부인원 {last_yesterday_people_data["내부인원"][0]})
- 최근일주일평균 : {last_week_people_total}명 (씨트립 {last_week_people_data["씨트립"][0]} + 한유망 {last_week_people_data["한유망"][0]} + 유학생 {last_week_people_data["유학생"][0]} + 내부인원 {last_week_people_data["내부인원"][0]})
- 최근한달평균 : {last_month_people_total}명 (씨트립 {last_month_people_data["씨트립"][0]} + 한유망 {last_month_people_data["한유망"][0]} + 유학생 {last_month_people_data["유학생"][0]} + 내부인원 {last_month_people_data["내부인원"][0]})

=> 지난요일 {weekday_to_string(day_week)} 대비 : {variance(standard_people_total, last_yesterday_people_total)} (씨트립 {diff_people_ctrip} | 한유망 {diff_people_hanyumang} | 유학생 {diff_people_student} | 내부인원 {diff_people_internal})
=> 최근일주일 대비 : {variance(standard_people_total, last_week_people_total)} (씨트립 {diff_people_ctrip2} | 한유망 {diff_people_hanyumang2} | 유학생 {diff_people_student2} | 내부인원 {diff_people_internal2})
=> 최근한달 대비 : {variance(standard_people_total, last_month_people_total)} (씨트립 {diff_people_ctrip3} | 한유망 {diff_people_hanyumang3} | 유학생 {diff_people_student3} | 내부인원 {diff_people_internal3})


2) 사용금액

- 기준일평균 : {standard_pay_total}원
- 전체{weekday_to_string(day_week)}평균 : {int_to_comma(week_pay_data.loc["평균":]["전체"][0])}원
- 지난{weekday_to_string(day_week)}평균 : {last_yesterday_pay_total}원
- 최근일주일평균 : {last_week_pay_total}원
- 최근한달평균 : {last_month_pay_total}원

=> 지난요일 {weekday_to_string(day_week)} 대비 : {variance(standard_pay_total, last_yesterday_pay_total)}
=> 최근일주일 대비 : {variance(standard_pay_total, last_week_pay_total)}
=> 최근한달 대비 : {variance(standard_pay_total, last_month_pay_total)}


3) 계약매장 사용여부
""")
is_contract = df_all_card[(df_all_card["결제날짜"] == yesterday) & (df_all_card["ktc_가맹여부"] == "1")]
if len(is_contract) == 0:
    print("계약매장 결제는 없습니다.")
else:
    print(f"계약매장 결제수는 {len(is_contract)}개 입니다.\n\n Store_ID | 매장명 | 결제수 | 결제인원 | 총결제금액")
    store_contract_data = store_contract(yesterday)
    for i in range(len(store_contract_data)):
        print(f"- {store_contract_data['store_id'][i]} | {store_contract_data['상점명'][i]} | {store_contract_data['결제수'][i]}건 | {store_contract_data['결제인원'][i]}명 | {int_to_comma(store_contract_data['총_결제금액'][i])}원")
    print("\n")

print("""
4) 분류별
""")

for cate in category_list:
    cate_this_data = category_all_data[category_all_data["네이버_대분류"] == cate].head(3).reset_index(drop=True)
    if len(cate_this_data) == 0:
        print(f"{cate} 결제는 없습니다.")
    else:
        print(f"{cate} 결제수 TOP{len(cate_this_data)}\n지역 | 결제건수 | 결제인원 | 총결제금액")
        for i in range(min(len(cate_this_data), 3)):
            area1 = cate_this_data["시/도"][i]
            area2 = cate_this_data["시/군/구"][i]
            print(f"{i+1}) {cate_this_data['지역'][i]} | {cate_this_data['결제수'][i]}건 | {cate_this_data['결제인원'][i]}명 | {int_to_comma(cate_this_data['총_결제금액'][i])}원 ({detail_category(df_all_card, yesterday, cate, area1, area2, cate_this_data['결제수'][i])})")
            
    print("\n")

NameError: name 'standard_people_total' is not defined